# Extraindo dados da Covid19 via API
### Fonte de Dados: https://brasil.io/dataset/covid19/
### Augusto SPINELLI

### Licença
Os dados dados convertidos estão sob a licença Creative Commons Attribution ShareAlike. Caso utilize os dados, cite a fonte original e quem tratou os dados, como: Fonte: Secretarias de Saúde das Unidades Federativas, dados tratados por Álvaro Justen e colaboradores/Brasil.IO. Caso compartilhe os dados, utilize a mesma licença.

#### Para criação do nosso modelo preditivo que tentará prever se um paciente covid irá a óbito ou não, utilizaremos o dataset de casos individualizados anonimos baseado no cep do paciente

## Data Munging

In [40]:
#importando dependencias
import pandas as pd
import numpy as np
import os

#Caminho dos arquivos RAW
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
data_path = os.path.join(raw_data_path, 'ceps2.csv')

#criando dataframes brutos/raw
raw_df = pd.read_csv(data_path)

#verificando informações do dataframe
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dt_notific                 50613 non-null  object 
 1   dt_inicio_sintomas         50603 non-null  object 
 2   bairro_resid__estadia      50640 non-null  object 
 3   ap_residencia_estadia      50640 non-null  float64
 4   evolu��o                   50640 non-null  object 
 5   dt_�bito                   4727 non-null   object 
 6   CEP                        50640 non-null  int64  
 7   Data_atualiza��o;OBJECTID  50640 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.1+ MB


In [41]:
#renomeando colunas com encoding errado
df = raw_df.rename(columns={"evolu��o": "evolucao", "dt_�bito": "dt_obito"})

In [42]:
#mapeando data de atualizacao de forma correta
def GetData(dataFull):
    d = dataFull.split(';')[0]
    return d

df['dt_registro'] = df['Data_atualiza��o;OBJECTID'].map(lambda x: GetData(x))

In [43]:
#removendo coluna inútil
del df['Data_atualiza��o;OBJECTID']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dt_notific             50613 non-null  object 
 1   dt_inicio_sintomas     50603 non-null  object 
 2   bairro_resid__estadia  50640 non-null  object 
 3   ap_residencia_estadia  50640 non-null  float64
 4   evolucao               50640 non-null  object 
 5   dt_obito               4727 non-null   object 
 6   CEP                    50640 non-null  int64  
 7   dt_registro            50640 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.1+ MB
